In [ ]:
!pip install aiohttp nest_asyncio ipywidgets
import aiohttp
import asyncio
import json
import os
from IPython.display import display
import ipywidgets as widgets

# Jupyter widgets for media type and range
media_type = widgets.RadioButtons(
    options=['Images', 'Videos', 'Both'],
    description='Media Type:',
    value='Images'
)
username_input = widgets.Text(description='Username:', value='')
range_type = widgets.RadioButtons(
    options=['All', 'Custom Range'],
    description='Range:',
    value='All'
)
start_offset = widgets.IntText(description='Start Offset:', value=0, disabled=True)
end_offset = widgets.IntText(description='End Offset:', value=100, disabled=True)
run_button = widgets.Button(description='Start Scraping')

# Enable/disable range inputs based on range_type
def on_range_change(change):
    start_offset.disabled = (change['new'] == 'All')
    end_offset.disabled = (change['new'] == 'All')
range_type.observe(on_range_change, names='value')

# Display widgets in Jupyter
try:
    from IPython import get_ipython
    if get_ipython() is not None:
        display(media_type, username_input, range_type, start_offset, end_offset, run_button)
except:
    pass

base_api = "https://coomer.su/api/v1"
service = "onlyfans"

async def fetch_posts(session, offset, user):
    url = f"{base_api}/{service}/user/{user}?o={offset}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    try:
        async with session.get(url, headers=headers) as response:
            if response.status != 200:
                print(f"Skipped posts at offset {offset}: HTTP {response.status}")
                return None
            return await response.json()
    except Exception as e:
        print(f"Skipped posts at offset {offset}: {e}")
        return None

async def download_image(session, url, path):
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
        async with session.get(url, headers=headers) as response:
            if response.status == 200:
                with open(path, 'wb') as f:
                    f.write(await response.read())
                print(f"Downloaded {os.path.basename(path)}")
                return True
            else:
                print(f"Skipped {url}: HTTP {response.status}")
                return False
    except Exception as e:
        print(f"Skipped {url}: {e}")
        return False

async def process_posts(session, posts, user, media_choice, output_dir):
    tasks = []
    for post in posts:
        # Check main file
        if "file" in post and post["file"]:
            file_path = post["file"].get("path")
            if file_path:
                is_image = file_path.endswith(('.jpg', '.jpeg', '.png', '.gif'))
                is_video = file_path.endswith(('.mp4', '.mov', '.avi', '.mkv'))
                if (media_choice == 'Images' and is_image) or (media_choice == 'Videos' and is_video) or (media_choice == 'Both'):
                    img_url = "https://coomer.su" + file_path
                    filename = post["file"].get("name") or img_url.split('/')[-1]
                    path = os.path.join(output_dir, filename)
                    tasks.append(download_image(session, img_url, path))

        # Check attachments
        for attach in post.get("attachments", []):
            attach_path = attach.get("path")
            if attach_path:
                is_image = attach_path.endswith(('.jpg', '.jpeg', '.png', '.gif'))
                is_video = attach_path.endswith(('.mp4', '.mov', '.avi', '.mkv'))
                if (media_choice == 'Images' and is_image) or (media_choice == 'Videos' and is_video) or (media_choice == 'Both'):
                    img_url = "https://coomer.su" + attach_path
                    filename = attach.get("name") or img_url.split('/')[-1]
                    path = os.path.join(output_dir, filename)
                    tasks.append(download_image(session, img_url, path))

    await asyncio.gather(*tasks, return_exceptions=True)

async def main():
    user = username_input.value.strip()
    if not user:
        print("Please enter a username!")
        return

    media_choice = media_type.value
    output_dir = f"{user}_{media_choice.lower()}"
    if range_type.value == 'Custom Range':
        output_dir += f"_range_{start_offset.value}_{end_offset.value}"
    else:
        output_dir += "_all"
    os.makedirs(output_dir, exist_ok=True)

    async with aiohttp.ClientSession() as session:
        if range_type.value == 'Custom Range':
            start = max(0, start_offset.value)
            end = max(start, end_offset.value)
            offset = start
            while offset <= end:
                posts = await fetch_posts(session, offset, user)
                if not posts:
                    break
                await process_posts(session, posts, user, media_choice, output_dir)
                offset += 50
        else:
            offset = 0
            while True:
                posts = await fetch_posts(session, offset, user)
                if not posts:
                    break
                await process_posts(session, posts, user, media_choice, output_dir)
                offset += 50

    print(f"All done, bro! Check the {output_dir} folder.")

# Run when button is clicked
def on_run_button_clicked(b):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            loop.create_task(main())
        else:
            loop.run_until_complete(main())
    except Exception as e:
        print(f"Error running scraper: {e}")

run_button.on_click(on_run_button_clicked)

# Run directly in terminal or if not using widgets
if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError as e:
        if "cannot be called from a running event loop" in str(e):
            import nest_asyncio
            nest_asyncio.apply()
            loop = asyncio.get_event_loop()
            loop.run_until_complete(main())
        else:
            raise
else:
    # Ensure Jupyter compatibility
    import nest_asyncio
    nest_asyncio.apply()